### Tutorial on how to use the data `resample` function of the s2s `time` module

We start by importing the required libraries and generating some an example Pandas Series and a DataFrame

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import s2spy.time

time_index = pd.date_range('20171020', '20211001', freq='15d')
random_data = np.random.random(len(time_index))
example_series = pd.Series(random_data, index=time_index)
example_dataframe = pd.DataFrame(example_series.rename('data1'))
example_dataframe['data2'] = example_dataframe['data1']

example_series

2017-10-20    0.076285
2017-11-04    0.603606
2017-11-19    0.065114
2017-12-04    0.564974
2017-12-19    0.201474
                ...   
2021-07-31    0.829086
2021-08-15    0.549621
2021-08-30    0.169399
2021-09-14    0.541165
2021-09-29    0.233646
Freq: 15D, Length: 97, dtype: float64

The DataFrame looks similar but has two named columns:

In [2]:
example_dataframe

,data1,data2
2017-10-20,0.076285,0.076285
2017-11-04,0.603606,0.603606
2017-11-19,0.065114,0.065114
2017-12-04,0.564974,0.564974
2017-12-19,0.201474,0.201474
...,...,...
2021-07-31,0.829086,0.829086
2021-08-15,0.549621,0.549621
2021-08-30,0.169399,0.169399
2021-09-14,0.541165,0.541165


To resample we need to set up an advent calendar with the anchor date and frequency. 
(Passing n_targets is optional, default is 1. This means i_interval = 0 is the target period.)

In [3]:
calendar = s2spy.time.AdventCalendar(anchor=(10, 15), freq='90d')
calendar.map_years(2018, 2020)

AdventCalendar(month=10, day=15, freq=90d, n_targets=1)

Next we pass the example data to the `resample` function. This requires a mapped calendar and the input data.


In [4]:
resampled_series = s2spy.time.resample(calendar, example_series)
resampled_series

,anchor_year,i_interval,interval,mean_data,target
0,2018,0,"(2018-07-17, 2018-10-15]",0.542593,True
1,2018,1,"(2018-04-18, 2018-07-17]",0.436143,False
2,2018,2,"(2018-01-18, 2018-04-18]",0.521101,False
3,2018,3,"(2017-10-20, 2018-01-18]",0.415118,False
4,2019,0,"(2019-07-17, 2019-10-15]",0.437233,True
5,2019,1,"(2019-04-18, 2019-07-17]",0.451927,False
6,2019,2,"(2019-01-18, 2019-04-18]",0.474606,False
7,2019,3,"(2018-10-20, 2019-01-18]",0.574506,False
8,2020,0,"(2020-07-17, 2020-10-15]",0.337540,True
9,2020,1,"(2020-04-18, 2020-07-17]",0.329619,False


As you see above, this generates a new DataFrame containing the data resampled for each interval, along with the corresponding lag time as well as the anchor year that the interval belongs to.

This works the same if you input a pandas DataFrame:

In [5]:
resampled_dataframe = s2spy.time.resample(calendar, example_dataframe)
resampled_dataframe

,anchor_year,i_interval,interval,data1,data2,target
0,2018,0,"(2018-07-17, 2018-10-15]",0.542593,0.542593,True
1,2018,1,"(2018-04-18, 2018-07-17]",0.436143,0.436143,False
2,2018,2,"(2018-01-18, 2018-04-18]",0.521101,0.521101,False
3,2018,3,"(2017-10-20, 2018-01-18]",0.415118,0.415118,False
4,2019,0,"(2019-07-17, 2019-10-15]",0.437233,0.437233,True
5,2019,1,"(2019-04-18, 2019-07-17]",0.451927,0.451927,False
6,2019,2,"(2019-01-18, 2019-04-18]",0.474606,0.474606,False
7,2019,3,"(2018-10-20, 2019-01-18]",0.574506,0.574506,False
8,2020,0,"(2020-07-17, 2020-10-15]",0.337540,0.337540,True
9,2020,1,"(2020-04-18, 2020-07-17]",0.329619,0.329619,False


This works the same for an `xarray` `Dataset`:

In [6]:
import xarray as xr

time_index = pd.date_range('20171020', '20211001', freq='15d')

np.random.seed(0)
temperature = 15 + 8 * np.random.randn(2, 2, len(time_index))
precipitation = 10 * np.random.rand(2, 2, len(time_index))

lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]


ds = xr.Dataset(
    data_vars=dict(
        temperature=(["x", "y", "time"], temperature),
        precipitation=(["x", "y", "time"], precipitation),
    ),
    coords=dict(
        lon=(["x", "y"], lon),
        lat=(["x", "y"], lat),
        time=time_index,
    ),
    attrs=dict(description="Weather related data."),
)
ds

<xarray.Dataset>
Dimensions:        (x: 2, y: 2, time: 97)
Coordinates:
    lon            (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat            (x, y) float64 42.25 42.21 42.63 42.59
  * time           (time) datetime64[ns] 2017-10-20 2017-11-04 ... 2021-09-29
Dimensions without coordinates: x, y
Data variables:
    temperature    (x, y, time) float64 29.11 18.2 22.83 ... 1.746 7.116 3.225
    precipitation  (x, y, time) float64 5.25 7.506 3.335 ... 7.359 1.415 8.659
Attributes:
    description:  Weather related data.

As you see below, the `temperature` and `precipitation` variables can 
easily be resampled to a new index using the calendar specified intervals.

This index dimension has as coordinates the anchor year and the interval count.



Note: both the `temperature` and `precipitation` variables have kept their 
`lat` and `lon` coordinates.

In [7]:
ds_r = s2spy.time.resample(calendar, ds)
ds_r

<xarray.Dataset>
Dimensions:        (anchor_year: 3, i_interval: 4, bounds: 2, x: 2, y: 2)
Coordinates:
  * anchor_year    (anchor_year) int32 2018 2019 2020
  * i_interval     (i_interval) int64 0 1 2 3
    index          (anchor_year, i_interval) int64 0 1 2 3 4 5 6 7 8 9 10 11
    interval       (bounds, anchor_year, i_interval) datetime64[ns] 2018-07-1...
    lon            (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat            (x, y) float64 42.25 42.21 42.63 42.59
  * bounds         (bounds) object 'left' 'right'
    target         (i_interval) bool True False False False
Dimensions without coordinates: x, y
Data variables:
    temperature    (x, y, anchor_year, i_interval) float64 14.52 18.33 ... 12.32
    precipitation  (x, y, anchor_year, i_interval) float64 3.224 5.594 ... 5.705